## How To Perform an Echo Experiment

### Prerequisites
This guide assumes you have a configured `DeviceSetup` as well as `Qubit` objects with assigned parameters. Please see these guides (add links) if you need to create your setup and qubits for the first time. However, you can also run this notebook "as is" using an emulated session. If you are just getting started with the LabOne Q Applications Library, please don't hesitate to reach out to us at info@zhinst.com.

### Background
In this how-to guide, you'll perform a Hahn echo experiment to measure the transversal relaxation of a qubit using the `echo` experiment included in the LabOne Q Applications Library. 

In a previous how-to-guide, we already used Ramsey interferometry (link to ramsey how-to) to measure the transversal relaxation ( $T_2^*$ ). However, the Ramsey experiment is sensitive to quasi-static, low-frequency fluctuations. The Hahn echo protocol is less sensitive to quasi-static noise. 

#### Ramsey protocol

Let's start with a short summary of the Ramsey protocol: 

A $X_{\pi/2}$ (X90) pulse positions the Bloch vector on the equator (Fig. 1a). Typically, the carrier frequency of this pulse is slightly detuned from the qubit frequency. As a result, the Bloch vector will rotate around the z-axis. After waiting for a time $\tau$, a second $X_{\pi/2}$-pulse projects the Bloch vector back on to the z-axis. Repeated measurements are made to take an ensemble averaged estimate of the qubit polarization, as a function of $\tau$. The resulting oscillations feature an approximately exponential decay function with decaytime $T_2^*$. (from https://arxiv.org/pdf/1904.06560)

Here, the “ $^*$ ” indicates that the Ramsey experiment is sensitive to inhomogeneous broadening. That is, it is highly sensitive to quasi-static, low-frequency fluctuations that are constant within one experimental trial, but vary from trial to trial, e.g., due to $1/f$-type noise. These fluctuations cause a dephasing of the qubit state (Fig. 1b) during the time $\tau$. 

#### Hahn echo protocol

Now let's have a look at the Hahn echo protocol, which is less sensitive to quasi-static noise:

We perform the same protocol as for the Ramsey interferometry, but place one extra $Y_\pi$ (Y180) pulse in-between the two $X_{\pi/2}$ pulses (X90). The accumulated dephasing during the first period can be undone (refocused) in the second period by rotating the qubit state by 180 $^\circ$ in-between through an $Y_{\pi}$ pulse. The final $X_{\pi/2}$-pulse projects the Bloch vector back on to the z-axis. In summary the overall quasi-static contributions to dephasing are reduced, leaving an estimate $T_{2e}$ that is less sensitive to inhomogeneous broadening mechanisms.

The pulses are generally chosen to be resonant with the qubit transition for a Hahn echo, since any frequency detuning would be nominally refocused anyway.


![](../../../images/hahn_echo.svg "Hahn echo experiment to refocus quasi-static contributions to dephasing.")

### Imports

You'll start by importing the echo experiment from `laboneq_applications`, as well as `laboneq.simple` and a demo QPU and device setup to run in emulation mode.

In [ ]:
%load_ext autoreload
%autoreload 2

from laboneq.contrib.example_helpers.plotting.plot_helpers import plot_simulation
from laboneq.simple import *

from laboneq_applications.experiments import echo
from laboneq_applications.qpu_types.tunable_transmon.demo_qpus import demo_platform

# from laboneq_applications.experiments.options import TuneupExperimentOptions

### QPU and Device Setup

You'll generate six qubits with pre-defined parameters, as well as a `Device_Setup` consisting of a SHFQC+, HDAWG, and PQSC. If you already have your own `DeviceSetup` and qubits configured, you'll instead initialize the session using your setup.

In [ ]:
my_platform = demo_platform(6)

Then, you'll connect to the `Session`. Here we connect to an emulated one:

In [ ]:
session = Session(my_platform.setup)
session.connect(do_emulation=True)

### Running the Echo Workflow

You'll now make the experiment workflow and run the following lines of code. The delay parameter corresponds to the delay $\tau$ in the figure above and must be given as one list per qubit. In particular, the time between the end of the first X90 pulse and the beginning of the Y180 pulse is defined as delay/2 ( $\tau/2$ ). The same is true for the delay between the end of the Y180 pulse and the beginning of the second X90 pulse.

In [ ]:
# our qubits live here in the demo setup:
qubits = my_platform.qpu.qubits

my_workflow = echo.experiment_workflow(
    session=session,
    qpu=my_platform.qpu,
    qubits=[qubits[0], qubits[1]],
    delays=[[1e-6, 2e-6, 3e-6, 4e-6, 5e-6],[1e-6, 2e-6, 3e-6, 4e-6, 5e-6]],
)

my_results = my_workflow.run()

#### Output Simulation

You can also inspect the compiled experiment and plot the simulated output:

In [ ]:
compiled_echo = my_results.tasks["compile_experiment"].output
plot_simulation(compiled_echo, length=60e-6)

#### Inspecting the Source Code

You can inspect the source code of the `create_experiment` task defined in `echo` to see how the experiment pulse sequence is created:

In [ ]:
echo.create_experiment.src

### Changing the Options

We can give our Hahn echo experiment options. First, inspect what they currently are:

In [ ]:
my_new_opts = echo.experiment_workflow.options()
my_new_opts

Then provide new options. This time, you'll remove the calibration traces. Furthermore, instead of an y180 pulse in-between the x90 pulses, you'll play an x180 pulse instead.

In [ ]:
my_new_opts.use_cal_traces(value=False)
my_new_opts.refocus_qop("x180")
my_new_opts

#### Run the workflow with updated options

Now, run the workflow with new options and inspect the simulated output. You'll notice that the calibration traces are no longer there and the sequence is now `x90 - x180 - x90`. 

In [ ]:
my_new_workflow = echo.experiment_workflow(
    session=session,
    qpu=my_platform.qpu,
    qubits=[qubits[0], qubits[1]],
    delays=[[1e-6, 2e-6, 3e-6, 4e-6, 5e-6],[1e-6, 2e-6, 3e-6, 4e-6, 5e-6]],
    options=my_new_opts,
)

my_new_results = my_new_workflow.run()
new_compiled_echo = my_new_results.tasks["compile_experiment"].output
plot_simulation(new_compiled_echo, length=50e-6)

### Temporary settings

The qubit parameters are used to control the settings of pulses and instruments during the experiment. We can run the Hahn echo experiment with different settings by passing it a copy of the qubits with modified parameters:

In [ ]:
# Make a copy of the qubits
temp_qubits = my_platform.qpu.copy_qubits()
# Change the length of the drive pulses
temp_qubits[0].parameters.ge_drive_length = 1000e-9
temp_qubits[1].parameters.ge_drive_length = 1000e-9

my_new_workflow = echo.experiment_workflow(
    session=session,
    qpu=my_platform.qpu,
    qubits=[temp_qubits[0], temp_qubits[1]],  # pass temporary qubits
    delays=[[1e-6, 2e-6, 3e-6, 4e-6, 5e-6],[1e-6, 2e-6, 3e-6, 4e-6, 5e-6]],
    options=my_new_opts,
)

my_new_results = my_new_workflow.run()
new_compiled_echo = my_new_results.tasks["compile_experiment"].output
plot_simulation(new_compiled_echo, length=10e-6)

Great! You've now run your Hahn echo experiment. Check out these other experiments to keep characterizing your qubits:

In [ ]:
# TODO: Add experiment links
# TODO: Add Analysis